In [ ]:
from dotenv import load_dotenv
from langchain_community.utilities import GoogleSerperAPIWrapper
import requests
import os
from typing import TypedDict,Annotated
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START,END
from langchain.agents import Tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display
import gradio as gr
from langgraph.checkpoint.memory import MemorySaver


In [ ]:
load_dotenv(override=True)

### Asynchronous Langgraph

In [ ]:
class State(TypedDict):
    message:Annotated[list,add_messages]
graph_builder = StateGraph(State)


In [ ]:
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_user = os.getenv("PUSHOVER_USER")
pushover_url = "https://api.pushover.net/1/messages.json"

def push(text: str):
    """Send a push notification to the user"""
    requests.post(pushover_url, data = {"token": pushover_token, "user": pushover_user, "message": text})

tool_push = Tool(
        name="send_push_notification",
        func=push,
        description="useful for when you want to send a push notification"
    )

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
!pip install langchain-community playwright

In [ ]:
!playwright install


In [ ]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser

async_browser = create_async_playwright_browser(headless=False)
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()

In [ ]:
for tool in tools:
    print(f"{tool.name}:{tool}")

In [ ]:
tool_dict ={ tool.name:tool for tool in tools}

navigate_tool = tool_dict.get("navigate_browser")
extract_text_tool =tool_dict.get("extract_text")

await navigate_tool.arun({"url":"https://www.cnn.com"})
text = await extract_text_tool.arun({})

In [ ]:
import textwrap
print(textwrap.fill(text))

In [ ]:
all_tools = tools + [tool_push]

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(all_tools)

def chatbot(state:State):
    return {"messages":[llm_with_tools.invoke(state["message"])]}

In [ ]:
graph_builder.add_node("chatbot",chatbot)
graph_builder.add_node("tools",ToolNode(tools=all_tools))
graph_builder.add_conditional_edges("chatbot",tools_condition,"tools")
graph_builder.add_edge("tools","chatbot")
graph_builder.add_edge(START,"chatbot")
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable":{"thread_id":"10"}}
def chat(user_input:str, history):
    result = graph.invoke({"messages":[{"role":"user","content":user_input}]},config=config)
    return result["messages"][-1].content

gr.ChatInterface(chat, type="messages").launch(server_port=8080)
